# Capacity Ratio Tutorial

This tutorial demonstrates how to run the **Capacity Ratio Study** experiment, which investigates how asymmetric resource capacities influence agent specialisation patterns and system performance.

## Overview

The capacity ratio study examines how different resource capacity configurations affect agent learning dynamics and specialisation patterns. Asymmetric capacities create hierarchical resource environments where high-capacity resources may attract more agents, whilst low-capacity resources remain underutilised or attract late convergers.

This experiment tests the hypothesis that capacity asymmetry drives predictable agent specialisation patterns, with agents naturally organising themselves according to resource capacity hierarchies through environmental feedback.

## Concept Explanation

### What are Capacity Ratios?

Capacity ratios refer to the relative capacity distribution across available resources in the system. These ratios determine how much load each resource can handle before experiencing congestion.

**Examples:**
- **Symmetric capacities**: [0.33, 0.33, 0.33] - Equal resource availability
- **Asymmetric capacities**: [0.6, 0.3, 0.1] - Unequal resource availability
- **Extreme asymmetry**: [0.8, 0.15, 0.05] - Highly hierarchical resources

### Why Study Capacity Ratios?

1. **Understanding how resource asymmetry influences agent specialisation**
2. **Identifying capacity-driven coordination patterns**
3. **Exploring hierarchical resource utilisation dynamics**
4. **Validating theoretical predictions about capacity effects**
5. **Optimising system performance through capacity design**

### Theoretical Foundation

Asymmetric capacity configurations create natural hierarchies in the resource environment. High-capacity resources provide better performance and attract more agents, whilst low-capacity resources may remain underutilised or serve as fallback options.

Capacity-driven specialisation can:
- **Create predictable agent organisation patterns**
- **Improve system efficiency through strategic resource allocation**
- **Enable hierarchical coordination without explicit communication**
- **Optimise resource utilisation based on capacity constraints**

### Key Research Questions

1. Do asymmetric capacities drive predictable specialisation patterns?
2. How do capacity hierarchies influence agent convergence timing?
3. What is the relationship between capacity asymmetry and performance?
4. Can capacity design be used to engineer desired coordination?
5. How do agents adapt to different capacity configurations?

### Expected Outcomes

- High-capacity resources should attract more agents
- Capacity-utilisation correlations should be positive
- Asymmetric configurations should improve system performance
- Hierarchical specialisation patterns should emerge
- Convergence timing should reflect capacity preferences

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime
import sys
import os
from typing import List, Tuple

# Add the parent directory to the path to import the experiment modules
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

from resource_allocation_sim.experiments.capacity_ratio_study import CapacityRatioStudy, run_capacity_ratio_study
from resource_allocation_sim.utils.config import Config
from resource_allocation_sim.evaluation.metrics import calculate_entropy, calculate_convergence_speed
from resource_allocation_sim.visualisation.plots import plot_convergence_comparison

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Libraries imported successfully!")

## Parameter Customisation

Now let's set up the experiment parameters. You can customise these values to explore different scenarios.

In [ ]:
# System Configuration
print("=== SYSTEM CONFIGURATION ===")
num_agents = int(input("Number of agents (default: 10): ") or "10")
num_resources = int(input("Number of resources (default: 3): ") or "3")
num_iterations = int(input("Number of iterations (default: 1000): ") or "1000")
num_replications = int(input("Number of replications (default: 50): ") or "50")

# Learning Parameters
print("\n=== LEARNING PARAMETERS ===")
weight = float(input("Learning rate (weight) (default: 0.3): ") or "0.3")

# Capacity Configuration Selection
print("\n=== CAPACITY CONFIGURATION SELECTION ===")
print("Choose capacity configuration method:")
print("1. Use all predefined configurations")
print("2. Select specific predefined configurations")
print("3. Create custom capacity configurations")
print("4. Mix predefined and custom configurations")

capacity_choice = input("Enter choice (1-4, default: 1): ") or "1"

# Convergence Parameters
print("\n=== CONVERGENCE PARAMETERS ===")
convergence_threshold_entropy = float(input("Convergence entropy threshold (default: 0.1): ") or "0.1")
convergence_threshold_max_prob = float(input("Convergence max probability threshold (default: 0.9): ") or "0.9")

# Output Configuration
print("\n=== OUTPUT CONFIGURATION ===")
output_dir = input("Output directory (default: results/capacity_ratio_tutorial): ") or "results/capacity_ratio_tutorial"

print("\n=== EXPERIMENT CONFIGURATION SUMMARY ===")
print(f"Number of agents: {num_agents}")
print(f"Number of resources: {num_resources}")
print(f"Number of iterations: {num_iterations}")
print(f"Number of replications: {num_replications}")
print(f"Learning rate (weight): {weight}")
print(f"Capacity configuration selection: {capacity_choice}")
print(f"Convergence entropy threshold: {convergence_threshold_entropy}")
print(f"Convergence max probability threshold: {convergence_threshold_max_prob}")
print(f"Output directory: {output_dir}")

In [ ]:
# Handle capacity configuration selection
if capacity_choice == "1":
    # Use all predefined configurations
    capacity_configs = [
        [0.33, 0.33, 0.33],  # Symmetric baseline
        [0.5, 0.3, 0.2],     # Moderate asymmetry 1
        [0.4, 0.4, 0.2],     # Moderate asymmetry 2
        [0.6, 0.3, 0.1],     # High asymmetry 1
        [0.7, 0.2, 0.1],     # High asymmetry 2
        [0.5, 0.25, 0.25],   # Single dominant
        [0.8, 0.15, 0.05],   # Extreme asymmetry
        [0.45, 0.45, 0.1],   # Two dominant
        [0.6, 0.25, 0.15],   # Graduated hierarchy
        [0.55, 0.35, 0.1]    # Strong binary
    ]
    print(f"Using all {len(capacity_configs)} predefined configurations")
    
elif capacity_choice == "2":
    # Select specific predefined configurations
    predefined_configs = [
        [0.33, 0.33, 0.33],  # Symmetric baseline
        [0.5, 0.3, 0.2],     # Moderate asymmetry 1
        [0.4, 0.4, 0.2],     # Moderate asymmetry 2
        [0.6, 0.3, 0.1],     # High asymmetry 1
        [0.7, 0.2, 0.1],     # High asymmetry 2
        [0.5, 0.25, 0.25],   # Single dominant
        [0.8, 0.15, 0.05],   # Extreme asymmetry
        [0.45, 0.45, 0.1],   # Two dominant
        [0.6, 0.25, 0.15],   # Graduated hierarchy
        [0.55, 0.35, 0.1]    # Strong binary
    ]
    
    print("\nAvailable predefined configurations:")
    for i, config in enumerate(predefined_configs, 1):
        print(f"{i:2d}. [{config[0]:.2f}, {config[1]:.2f}, {config[2]:.2f}]")
    
    selection = input("Enter configuration numbers (comma-separated, e.g., 1,3,5): ")
    indices = [int(x.strip()) - 1 for x in selection.split(',')]
    capacity_configs = [predefined_configs[i] for i in indices if 0 <= i < len(predefined_configs)]
    print(f"Selected {len(capacity_configs)} configurations")
    
elif capacity_choice == "3":
    # Create custom capacity configurations
    print("\n=== CUSTOM CAPACITY CONFIGURATIONS ===")
    print("You can create custom capacity configurations in several ways:")
    print("1. Manual entry of capacity values")
    print("2. Generate configurations with specific asymmetry levels")
    print("3. Create configurations based on resource types")
    
    custom_choice = input("Choose custom configuration type (1-3): ")
    
    if custom_choice == "1":
        print(f"Enter capacity values for each configuration")
        print(f"Example for {num_resources} resources: 0.6,0.3,0.1")
        capacity_configs = []
        while True:
            probs_str = input("Capacity values (comma-separated, or 'done' to finish): ")
            if probs_str.lower() == 'done':
                break
            capacities = [float(x.strip()) for x in probs_str.split(',')]
            # Normalise to sum to 1
            total = sum(capacities)
            capacities = [cap/total for cap in capacities]
            capacity_configs.append(capacities)
    
    elif custom_choice == "2":
        print("Generate configurations with specific asymmetry levels")
        num_configs = int(input("Number of configurations to generate (default: 5): ") or "5")
        asymmetry_levels = input("Asymmetry levels (comma-separated, e.g., 0.1,0.2,0.3): ")
        levels = [float(x.strip()) for x in asymmetry_levels.split(',')]
        
        capacity_configs = []
        for level in levels:
            # Create configuration with specified asymmetry
            base = (1.0 - level) / num_resources
            dominant = base + level
            config = [dominant] + [base] * (num_resources - 1)
            # Shuffle to create different patterns
            np.random.shuffle(config)
            capacity_configs.append(config)
    
    else:  # custom_choice == "3"
        print("Create configurations based on resource types")
        print("1. High-capacity primary resource")
        print("2. Balanced secondary resources")
        print("3. Low-capacity backup resources")
        
        primary_ratio = float(input("Primary resource capacity ratio (default: 0.6): ") or "0.6")
        secondary_ratio = float(input("Secondary resource capacity ratio (default: 0.3): ") or "0.3")
        backup_ratio = 1.0 - primary_ratio - secondary_ratio
        
        if backup_ratio >= 0:
            capacity_configs = [[primary_ratio, secondary_ratio, backup_ratio]]
        else:
            print("Invalid ratios - using default")
            capacity_configs = [[0.6, 0.3, 0.1]]
    
    print(f"Created {len(capacity_configs)} custom configurations")
    
else:  # capacity_choice == "4"
    # Mix predefined and custom configurations
    print("\n=== MIXED CONFIGURATION APPROACH ===")
    print("This will combine predefined configurations with custom ones.")
    
    # Start with predefined
    capacity_configs = [
        [0.33, 0.33, 0.33],  # Symmetric baseline
        [0.5, 0.3, 0.2],     # Moderate asymmetry 1
        [0.6, 0.3, 0.1],     # High asymmetry 1
        [0.8, 0.15, 0.05],   # Extreme asymmetry
    ]
    
    # Add custom configurations
    print("\nNow add custom configurations:")
    print("1. Manual entry of capacity values")
    print("2. Generate configurations with specific asymmetry levels")
    
    custom_choice = input("Choose custom configuration type (1-2): ")
    
    if custom_choice == "1":
        print(f"Enter capacity values for each configuration")
        print(f"Example for {num_resources} resources: 0.6,0.3,0.1")
        while True:
            probs_str = input("Capacity values (comma-separated, or 'done' to finish): ")
            if probs_str.lower() == 'done':
                break
            capacities = [float(x.strip()) for x in probs_str.split(',')]
            # Normalise to sum to 1
            total = sum(capacities)
            capacities = [cap/total for cap in capacities]
            capacity_configs.append(capacities)
    
    else:  # custom_choice == "2"
        print("Generate configurations with specific asymmetry levels")
        asymmetry_levels = input("Asymmetry levels (comma-separated, e.g., 0.1,0.2,0.3): ")
        levels = [float(x.strip()) for x in asymmetry_levels.split(',')]
        
        for level in levels:
            # Create configuration with specified asymmetry
            base = (1.0 - level) / num_resources
            dominant = base + level
            config = [dominant] + [base] * (num_resources - 1)
            # Shuffle to create different patterns
            np.random.shuffle(config)
            capacity_configs.append(config)
    
    print(f"Using {len(capacity_configs)} mixed configurations")

print("\nFinal capacity configurations:")
for i, config in enumerate(capacity_configs):
    print(f"  {i+1}. [{config[0]:.2f}, {config[1]:.2f}, {config[2]:.2f}])

## Running the Experiment

Now let's run the capacity ratio study experiment with the configured parameters.

In [ ]:
# Create output directory
output_path = Path(output_dir)
output_path.mkdir(parents=True, exist_ok=True)

print("=== RUNNING CAPACITY RATIO STUDY EXPERIMENT ===")
print(f"Creating CapacityRatioStudy...")

# Create and run the experiment
study = CapacityRatioStudy(
    capacity_configurations=capacity_configs,
    results_dir=output_dir,
    experiment_name="capacity_ratio_tutorial"
)

# Override base config with user parameters
study.base_config.num_agents = num_agents
study.base_config.num_resources = num_resources
study.base_config.num_iterations = num_iterations
study.base_config.weight = weight

print(f"Running experiment with {len(capacity_configs)} capacity configurations...")
print(f"Replications per configuration: {num_replications}")
print("This may take several minutes depending on the number of configurations and replications...")

# Run experiment
full_results = study.run_experiment(num_episodes=num_replications)

# Convert results to expected format
study.results = []
converted_results = []
for config_result in full_results['results']:
    config_params = config_result['config_params']
    for episode_result in config_result['episode_results']:
        converted_result = {
            'config_params': config_params,
            'simulation_results': episode_result,
            'replication_id': episode_result['episode']
        }
        study.results.append(converted_result)
        converted_results.append(converted_result)

print("Experiment completed successfully!")

## Generating Analysis and Plots

Now let's generate comprehensive analysis and create visualisations to understand the results.

In [ ]:
print("=== GENERATING ANALYSIS AND PLOTS ===")

# Temporarily set converted_results for analysis
study.converted_results = converted_results

# Generate detailed analysis
print("Generating detailed statistical analysis...")
analysis_results = study.perform_comprehensive_analysis()

# Create comprehensive plots
print("Creating comprehensive visualisations...")
plots_dir = f"{output_dir}/plots"
plot_files = study.create_comprehensive_plots(plots_dir)

print(f"Generated {len(plot_files)} plot files:")
for plot_file in plot_files:
    print(f"  - {plot_file}")

# Save analysis results
print("Saving analysis results...")
study.save_analysis_results(output_dir)

# Generate hypothesis report
print("Generating hypothesis evaluation report...")
report = study.generate_capacity_report()

# Save report
report_path = f"{output_dir}/capacity_ratio_hypothesis_report.txt"
with open(report_path, 'w') as f:
    f.write(report)

print(f"Hypothesis evaluation report saved to: {report_path}")

## Key Findings Summary

Let's examine the key findings from the experiment.

In [ ]:
print("=== KEY FINDINGS SUMMARY ===")

if 'hypothesis_support' in analysis_results:
    hyp_support = analysis_results['hypothesis_support']
    overall_support = hyp_support.get('overall_support', 'unknown')
    print(f"Overall Hypothesis Support: {overall_support.upper()}")
    
    evidence = hyp_support.get('evidence_strength', {})
    if 'capacity_correlation' in evidence:
        corr_info = evidence['capacity_correlation']
        print(f"Capacity-Utilisation Correlation: {corr_info['value']:.3f} ({corr_info['strength']})")
    
    if 'hierarchy_consistency' in evidence:
        hier_info = evidence['hierarchy_consistency']
        print(f"Hierarchy Consistency: {hier_info['value']:.3f} ({hier_info['strength']})")
    
    if 'specialisation_strength' in evidence:
        spec_info = evidence['specialisation_strength']
        print(f"Specialisation Index: {spec_info['value']:.3f} ({spec_info['strength']})")

if 'performance_analysis' in analysis_results:
    perf_analysis = analysis_results['performance_analysis']
    if 'total_costs' in perf_analysis:
        cost_data = perf_analysis['total_costs']
        best_config = min(cost_data.keys(), key=lambda k: np.mean(cost_data[k]))
        best_cost = np.mean(cost_data[best_config])
        print(f"\nBest performing configuration: {best_config.replace('_', ', ')}")
        print(f"  - Mean cost: {best_cost:.3f}")

# Display some key statistics
if 'capacity_correlation_analysis' in analysis_results:
    corr_analysis = analysis_results['capacity_correlation_analysis']
    if 'correlations' in corr_analysis:
        correlations = corr_analysis['correlations']
        mean_corr = np.mean(list(correlations.values()))
        print(f"\nMean capacity-utilisation correlation: {mean_corr:.3f}")
        
        # Show top correlations
        sorted_corrs = sorted(correlations.items(), key=lambda x: x[1], reverse=True)
        print("\nTop 3 capacity-utilisation correlations:")
        for i, (config, corr) in enumerate(sorted_corrs[:3]):
            print(f"  {i+1}. {config}: {corr:.3f}")

## Visualising Results

Let's display some key visualisations from the experiment.

In [ ]:
# Display key plots inline
print("=== KEY VISUALISATIONS ===")

# Load and display capacity correlation analysis
corr_plot_path = f"{plots_dir}/capacity_correlation_analysis.png"
if os.path.exists(corr_plot_path):
    print("\n1. Capacity-Utilisation Correlation Analysis:")
    from IPython.display import Image
    display(Image(filename=corr_plot_path))

# Load and display hierarchy analysis
hierarchy_plot_path = f"{plots_dir}/hierarchy_analysis.png"
if os.path.exists(hierarchy_plot_path):
    print("\n2. Hierarchical Specialisation Patterns:")
    display(Image(filename=hierarchy_plot_path))

# Load and display performance analysis
perf_plot_path = f"{plots_dir}/performance_analysis.png"
if os.path.exists(perf_plot_path):
    print("\n3. System Performance Across Configurations:")
    display(Image(filename=perf_plot_path))

# Load and display ternary specialisation comparison
ternary_plot_path = f"{plots_dir}/ternary_specialisation_comparison.png"
if os.path.exists(ternary_plot_path):
    print("\n4. Agent Specialisation in Ternary Space:")
    display(Image(filename=ternary_plot_path))

# Load and display statistical summary
stats_plot_path = f"{plots_dir}/statistical_summary.png"
if os.path.exists(stats_plot_path):
    print("\n5. Statistical Summary and Hypothesis Evaluation:")
    display(Image(filename=stats_plot_path))

## Detailed Analysis

Let's examine some detailed aspects of the results.

In [ ]:
# Detailed capacity correlation analysis
if 'capacity_correlation_analysis' in analysis_results:
    corr_analysis = analysis_results['capacity_correlation_analysis']
    
    print("=== CAPACITY-UTILISATION CORRELATION ANALYSIS ===")
    
    if 'correlations' in corr_analysis:
        correlations = corr_analysis['correlations']
        
        print("\nCorrelation values by configuration:")
        for config, corr in correlations.items():
            strength = "Strong" if corr > 0.7 else "Moderate" if corr > 0.4 else "Weak"
            print(f"  {config}: {corr:.3f} ({strength})")
        
        mean_corr = np.mean(list(correlations.values()))
        std_corr = np.std(list(correlations.values()))
        print(f"\nOverall correlation statistics:")
        print(f"  Mean: {mean_corr:.3f}")
        print(f"  Standard deviation: {std_corr:.3f}")
        print(f"  Range: {min(correlations.values()):.3f} - {max(correlations.values()):.3f}")
    
    if 'statistical_tests' in corr_analysis:
        stats = corr_analysis['statistical_tests']
        print(f"\nStatistical significance:")
        print(f"  T-test statistic: {stats.get('t_statistic', 'N/A'):.3f}")
        print(f"  P-value: {stats.get('p_value', 'N/A'):.6f}")
        print(f"  Significant: {stats.get('significant', 'N/A')}")

# Performance analysis
if 'performance_analysis' in analysis_results:
    perf_analysis = analysis_results['performance_analysis']
    
    print("\n=== PERFORMANCE ANALYSIS ===")
    
    if 'total_costs' in perf_analysis:
        costs = perf_analysis['total_costs']
        
        print("\nSystem costs by configuration:")
        for config, cost_values in costs.items():
            mean_cost = np.mean(cost_values)
            std_cost = np.std(cost_values)
            print(f"  {config}: {mean_cost:.3f} ± {std_cost:.3f}")
        
        # Find best and worst configurations
        config_means = {config: np.mean(values) for config, values in costs.items()}
        best_config = min(config_means, key=config_means.get)
        worst_config = max(config_means, key=config_means.get)
        
        print(f"\nBest configuration: {best_config} (cost: {config_means[best_config]:.3f})")
        print(f"Worst configuration: {worst_config} (cost: {config_means[worst_config]:.3f})")
        
        if best_config != worst_config:
            improvement = ((config_means[worst_config] - config_means[best_config]) / config_means[worst_config]) * 100
            print(f"Performance improvement: {improvement:.1f}%")

# Specialisation analysis
if 'agent_specialisation_analysis' in analysis_results:
    spec_analysis = analysis_results['agent_specialisation_analysis']
    
    print("\n=== AGENT SPECIALISATION ANALYSIS ===")
    
    if 'specialisation_indices' in spec_analysis:
        spec_indices = spec_analysis['specialisation_indices']
        
        print("\nSpecialisation indices by configuration:")
        for config, indices in spec_indices.items():
            mean_spec = np.mean(indices)
            std_spec = np.std(indices)
            print(f"  {config}: {mean_spec:.3f} ± {std_spec:.3f}")
        
        overall_mean = np.mean([np.mean(indices) for indices in spec_indices.values()])
        print(f"\nOverall mean specialisation index: {overall_mean:.3f}")

## Output Files and Structure

Let's examine what files were generated and their structure.

All results are saved in: **`<output_path>`**

## File structure:
<output_path>/ \
├── plots/ \
│ ├── capacity_correlation_analysis.png \
│ ├── hierarchy_analysis.png \
│ ├── performance_analysis.png \
│ ├── ternary_specialisation_comparison.png \
│ └── statistical_summary.png \
├── capacity_ratio_raw_data.csv \
├── analysis_results.json \
└── capacity_ratio_hypothesis_report.txt 

## File descriptions:

- **`plots/`**: All generated visualisations  
- **`capacity_ratio_raw_data.csv`**: Raw experimental data  
- **`analysis_results.json`**: Statistical analysis results  
- **`capacity_ratio_hypothesis_report.txt`**: Detailed hypothesis evaluation

## Key plots explained:

- **`capacity_correlation_analysis.png`**: Capacity-utilisation correlation analysis  
- **`hierarchy_analysis.png`**: Hierarchical specialisation patterns  
- **`performance_analysis.png`**: System performance across configurations  
- **`ternary_specialisation_comparison.png`**: Agent specialisation in ternary space  
- **`statistical_summary.png`**: Statistical tests and hypothesis evaluation


## Hypothesis Evaluation Report

Let's examine the detailed hypothesis evaluation report.

In [ ]:
# Display the hypothesis evaluation report
report_path = f"{output_dir}/capacity_ratio_hypothesis_report.txt"
if os.path.exists(report_path):
    print("=== HYPOTHESIS EVALUATION REPORT ===")
    print()
    
    with open(report_path, 'r') as f:
        report_content = f.read()
    
    # Display the report in chunks for better readability
    lines = report_content.split('\n')
    chunk_size = 50
    
    for i in range(0, len(lines), chunk_size):
        chunk = lines[i:i+chunk_size]
        print('\n'.join(chunk))
        
        if i + chunk_size < len(lines):
            input("\nPress Enter to continue reading the report...")
else:
    print("Hypothesis evaluation report not found.")

## Tutorial Summary

Congratulations! You have successfully completed the Capacity Ratio Study tutorial. Here's what we've accomplished:

### What We Learned

1. **Capacity ratios** define how resources are distributed in the system
2. **Asymmetric capacities** create hierarchical resource environments
3. **Agent specialisation** is influenced by resource capacity patterns
4. **System performance** varies with capacity configuration
5. **Hierarchical coordination** emerges from capacity-driven feedback

### Key Insights

- High-capacity resources tend to attract more agents
- Capacity-utilisation correlations indicate specialisation patterns
- Asymmetric configurations can improve system performance
- Hierarchical specialisation emerges naturally from capacity constraints
- Capacity design can be used to engineer desired coordination

### Next Steps

You can now:
1. **Explore other tutorials** to understand different aspects of the system
2. **Modify parameters** to test different hypotheses
3. **Combine experiments** to create comprehensive studies
4. **Analyse results** in more detail using the generated data
5. **Extend the framework** with new capacity configurations

### Files Generated

All results are saved in your specified output directory, including:
- Raw experimental data
- Statistical analysis results
- Comprehensive visualisations
- Hypothesis evaluation report

Thank you for completing this tutorial!